In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
from IPython.display import Markdown, display, HTML

load_dotenv()
os.chdir(os.path.dirname(os.getcwd()))

df = pd.read_parquet('./data/splade.parquet')
print(f"df shape: {df.shape}")
df.head(1)

df shape: (1987, 24)


,index,id,citation,name,name_abbreviation,decision_date,court_id,court_name,court_slug,judges,attorneys,citations,url,head,body,name_contains_lm,body_contains_lm,year,context,context_citation,context_tokens,openai_embeddings,splade_embeddings,state
0,0,411690,154 Ill. 2d 90,"RICHARD R. JOHNSON, Plaintiff-Appellant and Cr...",Johnson v. Halloran,2000-01-13,8837,Illinois Appellate Court,ill-app-ct,[],"['Wolter, Beeman, Lynch & McIntyre, of Springf...","[{'type': 'official', 'cite': '312 Ill. App. 3...",https://api.case.law/v1/cases/411690/,"RICHARD R. JOHNSON, Plaintiff-Appellant and Cr...",JUSTICE HALL\r\ndelivered the opinion of the c...,False,True,2000,The public defender of Cook County was appoint...,154 Ill. 2d 90,1317,"[-0.0017778094625100493, -0.002360282698646187...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",New Mexico


In [2]:
query = """
How have courts interpreted the phrase 'sudden and accidental' with respect to water damage?
"""

In [3]:
from src.search.models import (
    QueryPlanConfig,
    SearchRequest,
    DocumentSource,
    BaseFilters,
    Tag
)
from src.search.query_expansion import segment_search_query
from src.search.query_planning import generate_query_filters, screen_query

DEFAULT_TOP_K = 40
DEFAULT_RERANK_TOP_K = 20
DEFAULT_MAX_LLM_CHUNKS = 10


def retrieval_preprocessing(
    search_request: SearchRequest,
    verbose: bool = False,
) -> QueryPlanConfig:
    
    tag_list = []
    
    query = search_request.query

    preset_filters = search_request.human_selected_filters or None
    
    if preset_filters:
        tag_list.extend(preset_filters.tags)

    query_screening = None
    if search_request.enable_query_screening:
        query_screening = screen_query(query=query)
        
    if search_request.enable_auto_detect_filters:
        predicted_filters = generate_query_filters(
            search_df=df,
            query=query,
            filter_fields=['state'],
            verbose=verbose,
        )
        pred_tags = []
        for tag in predicted_filters:
            pred_tag = Tag(
                tag_key=tag.filter_key,
                tag_value=tag.filter_value
            )
            pred_tags.append(pred_tag)
        
        tag_list.extend(pred_tags)
        
    if search_request.enable_auto_subquestions:
        n = str(query_screening.n_subquestions) if query_screening else "2 to 3"
        queries = segment_search_query(
            query,
            n,
        )
        
    return QueryPlanConfig(
        query=search_request.query,
        subquestions=queries,
        filters=tag_list,
        num_hits=DEFAULT_TOP_K,
        num_rerank=DEFAULT_RERANK_TOP_K,
        max_llm_filter_chunks=DEFAULT_MAX_LLM_CHUNKS,
        rerank=search_request.rerank,
        llm_chunk_filter=search_request.llm_chunk_filter,
        query_eval=query_screening if query_screening is not None else None,
        search_configs=search_request.search_configs,
    )


In [4]:
from src.agent.tools.semantic_search import SemanticSearchConfig
from src.agent.tools.splade_search import SPLADESearchConfig

DATA_PATH = "data/splade.parquet"

SEMANTIC_TEXT_COLUMN = "context"
SPLADE_TEXT_COLUMN = "context"

semantic_config = SemanticSearchConfig(
    data_path=DATA_PATH,
    text_column=SEMANTIC_TEXT_COLUMN,
)
keyword_config = SPLADESearchConfig(
    data_path=DATA_PATH,
    text_column=SEMANTIC_TEXT_COLUMN,
)

# Tag can be added manually &/or be generated by llm
state_tag = Tag(
    tag_key="state",
    tag_value="Texas"
)
base_filters = BaseFilters(
    source_type=DocumentSource.OPINIONS,
    document_set="sharepoint",
    tags=[state_tag],
)
search_request = SearchRequest(
    query=query,
    human_selected_filters=None,
    enable_query_screening=True,
    enable_auto_detect_filters=True,
    enable_auto_subquestions=True,
    rerank=False,
    llm_chunk_filter=False,
    search_configs=[semantic_config, keyword_config]
)
preprocessed_search_query = retrieval_preprocessing(
    search_request=search_request,
    verbose=True,
)

2024-05-04 21:27:08 - INFO - Schema shown to LLM: 
Name of each field, its type and unique values (up to 20):
* state (string);  Values - ['New Mexico' 'Nevada' 'Virginia' 'Vermont' 'Kansas' 'Maryland' 'Arkansas'
 'Massachusetts' 'West Virginia' 'Pennsylvania' 'Texas' 'Maine'
 'Mississippi' 'Hawaii' 'Iowa' 'Kentucky' 'Ohio' 'New Hampshire'
 'Wisconsin' 'North Dakota'], ... 30 more
        


In [5]:
preprocessed_search_query.model_dump()

{'query': "\nHow have courts interpreted the phrase 'sudden and accidental' with respect to water damage?\n",
 'filters': [{'tag_key': 'text',
   'tag_value': 'sudden and accidental water damage'}],
 'num_hits': 40,
 'rerank': False,
 'num_rerank': 20,
 'llm_chunk_filter': False,
 'max_llm_filter_chunks': 10,
 'subquestions': {'questions': [{'sub_question_topic': "Interpretation of 'Sudden and Accidental' in Water Damage Cases",
    'sub_question_query': "What is the judicial interpretation of the phrase 'sudden and accidental' in the context of water damage claims in insurance cases?",
    'sub_question_keywords': ['sudden and accidental',
     'water damage',
     'judicial interpretation',
     'insurance claims',
     'case law']}]},
 'query_eval': {'topic': "Interpretation of 'sudden and accidental' with respect to water damage",
  'question_quality': "The question is well-defined and focused on a specific legal concept, which is the interpretation of the phrase 'sudden and accide

In [6]:
preprocessed_search_query.filters[0].filter_dict

{'text': 'sudden and accidental water damage'}

In [7]:
preprocessed_search_query.subquestions.questions[0].sub_question_query

"What is the judicial interpretation of the phrase 'sudden and accidental' in the context of water damage claims in insurance cases?"

In [8]:
preprocessed_search_query.subquestions.questions[0].sub_question_keywords

['sudden and accidental',
 'water damage',
 'judicial interpretation',
 'insurance claims',
 'case law']

In [9]:
retrieval_res = preprocessed_search_query.subquestions.execute(
    preprocessed_search_query,
)

2024-05-04 21:27:18 - INFO - Read in df with shape: (1987, 24)
2024-05-04 21:27:18 - INFO - Read in df with shape: (1987, 24)
2024-05-04 21:27:19 - INFO - Using pre-computed 'context' embeddings from existing column: splade_embeddings
2024-05-04 21:27:19 - INFO - 

Thought: To answer this query, we need to explore how judicial decisions have defined and applied the phrase
'sudden and accidental' in the context of water damage claims. This involves looking at case law to
understand the criteria or conditions under which water damage is considered 'sudden and accidental'
as opposed to gradual or expected, which influences the outcome of insurance claims and liabilities.
Search topic: Interpretation of 'Sudden and Accidental' in Water Damage Cases
2024-05-04 21:27:19 - INFO - Running search using SemanticSearch: What is the judicial interpretation of the phrase 'sudden and accidental' in the context of water damage claims in insurance cases?
2024-05-04 21:27:19 - INFO - Running search usi

In [10]:
retrieval_res[0].to_pandas.head(2)

,context,index,id,citation,name,name_abbreviation,decision_date,court_id,court_name,court_slug,judges,attorneys,citations,url,head,body,name_contains_lm,body_contains_lm,year,context_citation,context_tokens,state,search_type,score
1675,Id. at 4. According to Plaintiff's insurance p...,2126,12526556,91 L. Ed. 2d 265,"Audrey SANKO, Plaintiff, v. ALLSTATE INSURANCE...",Sanko v. Allstate Ins. Co.,2018-05-18,9039,United States District Court for the Middle Di...,md-pa,[],"['Tammy Lee Clause, Attorney Tammy Lee Clause,...","[{'type': 'official', 'cite': '323 F. Supp. 3d...",https://api.case.law/v1/cases/12526556/,"Audrey SANKO, Plaintiff,\r\nv.\r\nALLSTATE INS...","Robert D. Mariani, United States District Judg...",False,True,2018,91 L. Ed. 2d 265,1442,Wyoming,vector,0.976563
584,"In the absence of clear Missouri law, the tria...",754,5454708,197 Ill. 2d 278,"EMERSON ELECTRIC COMPANY et al., Plaintiffs-Ap...",Emerson Electric Co. v. Aetna Casualty & Suret...,2004-08-30,8837,Illinois Appellate Court,ill-app-ct,"['McNULTY and McBRIDE, JJ., concur.']","['Stephan G. Weil, Mark H. Kolman, Keisha A. G...","[{'type': 'official', 'cite': '352 Ill. App. 3...",https://api.case.law/v1/cases/5454708/,"EMERSON ELECTRIC COMPANY et al., Plaintiffs-Ap...",JUSTICE GORDON\r\ndelivered the opinion of the...,False,True,2004,197 Ill. 2d 278,1285,Massachusetts,vector,0.955224


In [11]:
retrieval_res[0].metrics[:10]

[ChunkMetric(source='SemanticSearch', document_id='584', score=0.8762, rank=1),
 ChunkMetric(source='SemanticSearch', document_id='585', score=0.8621, rank=2),
 ChunkMetric(source='SemanticSearch', document_id='1671', score=0.8601, rank=3),
 ChunkMetric(source='SemanticSearch', document_id='1675', score=0.8596, rank=4),
 ChunkMetric(source='SemanticSearch', document_id='1672', score=0.8595, rank=5),
 ChunkMetric(source='SemanticSearch', document_id='1673', score=0.8593, rank=6),
 ChunkMetric(source='SemanticSearch', document_id='1674', score=0.8589, rank=7),
 ChunkMetric(source='SemanticSearch', document_id='1670', score=0.8586, rank=8),
 ChunkMetric(source='SemanticSearch', document_id='1955', score=0.8577, rank=9),
 ChunkMetric(source='SemanticSearch', document_id='115', score=0.851, rank=10)]

In [12]:
from src.agent.tools.utils import aget_fact_patterns_df, aget_context_evaluations_df
import nest_asyncio
nest_asyncio.apply()

rag_df = retrieval_res[0].to_pandas.drop_duplicates(subset=['name'], keep='first')

# prepped_context = await aget_fact_patterns_df(rag_df, 'context')

In [13]:
prepped_context = await aget_context_evaluations_df(
    df=rag_df,
    question=query,
    text_column='context',
    model_name='gpt-4-turbo',    
)
print(prepped_context['relevance_score'].tolist()[:5])

Processing context evaluations: 100%|██████████| 9/9 [00:25<00:00,  2.85s/it]

[9, 8, 7, 1, 1]


In [14]:
i = 0

display(Markdown(str(prepped_context['relevance_score'].tolist()[i])))
display(Markdown(prepped_context['summary_analysis'].tolist()[i]))

9

The excerpt discusses a dispute over an insurance claim related to water damage, specifically referencing a 'water damage clause' in the plaintiff's insurance policy. The clause excludes coverage for various types of water damage but includes a provision for 'sudden and accidental' damage caused by ‘fire, explosion, or theft resulting from' specific excluded water-related incidents. The plaintiff argues that the damage was due to a 'sudden and accidental explosion' caused by a sewage system failure which should be covered under the policy. The defendant contends that for the damage to be covered, the excluded water damage must precede and cause the explosion, which was not the case. The document refers to a prior ruling (Colella v. State Farm Fire & Cas. Co., 407 F. App'x 616 (3d Cir. 2011)) which found similar water damage exclusion clauses precluded coverage for insured claims, indicating a relevant judicial interpretation of 'sudden and accidental' within the context of water damage exclusions in insurance policies.

In [15]:
# Markdown(prepped_context['summary'].tolist()[i])

The Plaintiff's insurance policy with the Defendant includes a water damage exclusion clause that denies coverage for losses caused by various forms of water damage, including floods, sewer backups, and groundwater pressure—yet it makes an exception for 'sudden and accidental direct physical loss caused by fire, explosion, or theft resulting from such water damage. In a dispute, the Plaintiff asserts that damage to her home was caused by an explosion due to sewage pressure, a scenario she argues falls under the policy's exception. The Defendant refutes this claim, interpreting the policy language to mean the water damage must precede and cause the explosion, which is not the case according to the Defendant, as the explosion led to the damages. The legal contention centers on whether the Plaintiff's circumstances qualify for coverage under the policy's exception. The court relies on precedent set by Colella v. State Farm Fire & Cas. Co., which upheld a broad water damage exclusion clause, to analyze the policy's applicability to the Plaintiff's situation.

In [15]:
prepped_context.head(3)

,context,index,id,citation,name,name_abbreviation,decision_date,court_id,court_name,court_slug,judges,attorneys,citations,url,head,body,name_contains_lm,body_contains_lm,year,context_citation,context_tokens,state,search_type,score,summary_analysis,relevance_score
0,Id. at 4. According to Plaintiff's insurance p...,2126,12526556,91 L. Ed. 2d 265,"Audrey SANKO, Plaintiff, v. ALLSTATE INSURANCE...",Sanko v. Allstate Ins. Co.,2018-05-18,9039,United States District Court for the Middle Di...,md-pa,[],"['Tammy Lee Clause, Attorney Tammy Lee Clause,...","[{'type': 'official', 'cite': '323 F. Supp. 3d...",https://api.case.law/v1/cases/12526556/,"Audrey SANKO, Plaintiff,\r\nv.\r\nALLSTATE INS...","Robert D. Mariani, United States District Judg...",False,True,2018,91 L. Ed. 2d 265,1442,Wyoming,vector,0.976563,The excerpt discusses a dispute over an insura...,9
1,"In the absence of clear Missouri law, the tria...",754,5454708,197 Ill. 2d 278,"EMERSON ELECTRIC COMPANY et al., Plaintiffs-Ap...",Emerson Electric Co. v. Aetna Casualty & Suret...,2004-08-30,8837,Illinois Appellate Court,ill-app-ct,"['McNULTY and McBRIDE, JJ., concur.']","['Stephan G. Weil, Mark H. Kolman, Keisha A. G...","[{'type': 'official', 'cite': '352 Ill. App. 3...",https://api.case.law/v1/cases/5454708/,"EMERSON ELECTRIC COMPANY et al., Plaintiffs-Ap...",JUSTICE GORDON\r\ndelivered the opinion of the...,False,True,2004,197 Ill. 2d 278,1285,Massachusetts,vector,0.955224,The excerpt discusses legal principles surroun...,8
3,"Addison Insurance Co., 376 Ill. App. 3d at 88....",1275,4277921,144 Ill. 2d 64,"STONERIDGE DEVELOPMENT COMPANY, INC., et al., ...",Stoneridge Development Co. v. Essex Insurance,2008-05-06,8837,Illinois Appellate Court,ill-app-ct,[],"['Barbara I. Michaelides, Steven N. Novosad, a...","[{'type': 'official', 'cite': '382 Ill. App. 3...",https://api.case.law/v1/cases/4277921/,"STONERIDGE DEVELOPMENT COMPANY, INC., et al., ...",JUSTICE BOWMAN\r\ndelivered the opinion of the...,False,True,2008,144 Ill. 2d 64,1515,Minnesota,vector,0.782180,The excerpt primarily discusses legal definiti...,7


In [17]:
from src.agent.tools.utils import create_formatted_input, get_final_answer

formatted_input = create_formatted_input(
    df=prepped_context, 
    query=retrieval_res[0].sub_question, 
    text_column='summary',
    url_column='url',
    context_token_limit=6000
    )

In [18]:
Markdown(formatted_input)

[1] The central issues in this legal dispute involve the interpretation of insurance policy language, specifically the terms "sudden and accidental" in the context of environmental contamination at owned sites in Alabama and Louisiana. On appeal, plaintiffs argue that the trial court erroneously applied Missouri law instead of Illinois law, the latter of which interprets "sudden and accidental" to mean "unexpected and unintended." Moreover, plaintiffs contest the trial court's interpretation of "sudden" as "abrupt" under Missouri law and its disregard of evidence indicating abrupt and unintended pollutant releases could have contributed to the contamination. The procedural backdrop includes a summary judgment framework, which dictates that such a judgment is appropriate only when there's no genuine issue of material fact and the moving party is entitled to judgment as a matter of law. The order granting summary judgment is subject to de novo review, with appellate courts required to independently examine the evidence for and against the motion to ensure that reasonable divergent inferences from undisputed facts could not lead to a different conclusion. Additionally, this case touches upon the law of the case doctrine, emphasizing its role in ensuring consistency and finality in litigation, albeit with an acknowledgment of its flexibility under certain circumstances like intervening changes in law. The previous decision in Emerson I comes under scrutiny for its interpretation relevant to the pollution exclusion exception in insurance policies, particularly the defining moment of "discharge" or "release" that triggers said exception.
URL: https://api.case.law/v1/cases/5454708/

[2] The insurance policy's exclusions for coverage specifically include a variety of water damage scenarios, excluding coverage for losses caused by flood, surface water, sewer or drain backups, sump pump overflows, and subsurface water pressure or seepage. However, it does provide coverage for sudden and accidental physical loss due to fire, explosion, or theft resulting from such water-related issues. In a disputed claim, the plaintiff references a specific event where a sewage system explosion caused damage, arguing for coverage under the policy's exception for explosions. The defendant challenges this claim, asserting that the explosion must be a consequence of the excluded water damage, not a preceding event, to qualify for coverage. This interpretation dispute is set against the backdrop of summary judgment standards emphasizing the need for genuine disputes over material facts to oppose summary judgment effectively. Colella v. State Farm Fire & Cas. Co. is cited as a relevant case for interpreting similar water damage exclusion clauses.
URL: https://api.case.law/v1/cases/12526556/

[3] In a legal dispute between Varlen and its insurer, Liberty Mutual, regarding coverage for contamination damage at the LASI and Silvis sites, the core issue revolved around whether the chemical leaks or discharges were "sudden and accidental" as outlined in Liberty Mutual's policy. Varlen's primary evidence to support this claim was the expert testimony of geologist Daniel Rogers. Rogers posited that the contamination was sudden and accidental based on the characteristics of the concrete sump leak at the LASI site and the nature of diesel and solvent contamination at the Silvis site. However, the district court ruled Rogers's testimony as unreliable and speculative, not meeting the Federal Rule of Evidence 702 standards for expert testimony. This was based on the reasoning that Rogers's conclusions were not sufficiently grounded in reliable methods or principles. Consequently, the court granted Liberty Mutual's motion for summary judgment after striking Rogers's testimony, pointing out that Varlen failed to provide admissible evidence that a reasonable jury could rely on to find the contamination events were "sudden and accidental".
URL: https://api.case.law/v1/cases/12521187/

[4] The case discusses the interpretation of terms within a Commercial General Liability (CGL) policy, specifically focusing on 'property damage' and 'occurrence,' and their implications for coverage. 'Property damage' is defined within the policy as either physical injury to tangible property or loss of use of tangible property that is not physically injured. An 'occurrence' is defined as an accident, which includes continuous or repeated exposure to substantially the same general harmful conditions. However, the policy lacks a definition for 'accident,' leading to ambiguity. Courts have therefore emphasized a liberal construction of 'accident' in favor of coverage, interpreting it as an unforeseen and unplanned event, or an event resulting from carelessness or ignorance where the person responsible did not intend or expect the resulting harm. The specific case highlights a scenario where the plaintiffs allege physical damage to their property due to negligent actions, claiming this damage falls under the policy's broad definition of 'property damage.' The court must decide whether this damage resulted from an 'occurrence' as per the policy's definition. Early arguments suggest that the negligence leading to property damage constitutes an 'occurrence' because the actions leading to the damage were not intended or expected by the defendant, aligning with the judicial interpretation of an 'accident.' The text also touches upon the potential for the CGL policy to be overly broad, suggesting it might cover scenarios typically excluded, akin to a performance bond.
URL: https://api.case.law/v1/cases/4268691/

[5] The plaintiff in a declaratory judgment action bears the burden of proof, as established by Addison Insurance Co., emphasizing the appellees' initial responsibility in such cases. The dispute centers on whether the damage to the Walskis’ home falls under policy coverage, challenged on grounds it was not caused by an “occurrence” and that contractual liability for economic damages does not equate to “property damage.” The policy definition of “occurrence” as an accident or continuous exposure to harmful conditions is scrutinized. Various Illinois appellate court decisions and definitions from Black’s Law Dictionary delineate “accident” as unforeseen, injurious occurrences not expected in the usual course of events. The legal dialogue delves into whether the natural and ordinary consequences of an act, or actions intentionally done with unintended results, constitute an accident. Carr versus Stoneridge cases illuminate differing interpretations based on the nature of claims (negligence versus breach of contract), influenced by precedent from Supreme Court and appellate court rulings, including nuanced discussions on the intent behind actions leading to accidents. The complication around what defines an “accident” underscores the nuanced legal interpretations of policy language and the implications for coverage in cases of property damage claims.
URL: https://api.case.law/v1/cases/4277921/

[6] Visalia's legal battle revolves around disputes concerning environmental contaminant cleanup costs, highlighting the non-relevance of ‘sudden and accidental’ releases due to explicit policy exclusions. Visalia’s reliance on affirmative defenses against RLI, including waiver, estoppel, and fraud, is further nuanced by the addition of 'unclean hands' and 'forfeiture' defenses in their Amended Answer. The parties debated on the appropriate standard for pleading affirmative defenses, referencing a lack of clarity between the 'fair notice' and the heightened 'plausibility' standards established by federal case law. The court dismissed certain requests for judicial notice as irrelevant, underscoring the focus on the legal tenability and the strategic approaches taken by both parties within the litigation framework. The discussion also dabbles in the procedural postings and the specific legal precedents guiding the pleading standards for affirmative defenses within the Ninth Circuit jurisdiction, reflecting broader implications on case strategy and judicial discretion.
URL: https://api.case.law/v1/cases/12524005/

[7] Williams, 143 Ill. App. 3d at 111 involves a dispute over insurance coverage for water damage to property. The core issue revolves around interpreting the policy to determine coverage eligibility. Critical to the case are factual findings regarding the cause of the water damage, with the court finding it resulted from rain collecting on the roof leading to a partial collapse. This finding contrasts with testimonies suggesting the damage was due to normal wear and tear and that no roof repairs had been observed or conducted during the relevant storage period. Notably, the admission of roof repairs between July 1997 and November 1998 supported the court’s decision. The case was marked by conflicting evidence, necessitating multiple factual findings, specifically 15, compared to only three conclusions of law. The appellate court found the circuit court’s order to strike the defendant’s jury demand as an abuse of discretion, emphasizing the need for a jury to evaluate witness credibility and factual causes of the water damage for proper policy coverage determination. Consequently, it was remanded for a new trial. The appellate review also addressed the propriety of denying the defendant's motion for summary judgment, highlighting the existence of factual disputes and criticizing the defendant's contradictory stances on the presence of factual issues. The decision underscores principles of judgment and jury involvement in the context of insurance dispute resolution and factual ambiguity.
URL: https://api.case.law/v1/cases/4267133/

[8] Hugo's Estate argues for a broad interpretation of "occurrence" under Kentucky law to include unintentional injuries from negligent acts, like exposure to secondhand smoke, advocating that such mishaps not planned or known to cause harm should be considered accidents covered by insurance. It disputes the claim of "bodily injury" arising from "physical abuse" as defined in the policy, suggesting that the common understanding of abuse does not align with Hugo's unknowingly harmful smoking around his children. The legal backdrop includes the summary judgment standard, guiding the court to resolve cases without a trial when there's no factual dispute and one party is entitled to judgment as a legal right. Crucially, Kentucky law, chosen based on the most significant relationship to the case -- particularly insurance matters -- emphasizes that insurance contracts are interpreted by the court, focusing on the mutual understanding of the contract terms at inception. Insurers have a duty to defend if claims could potentially fall under policy coverage unless clearly excluded. The text suggests a heavy reliance on statutory interpretation, precedent, and the specific language of the insurance contract in determining the obligations of the insurer and the rights of the insured.
URL: https://api.case.law/v1/cases/12531212/

[9] A legal dispute arose when plaintiffs filed a claim with Union for water damage to their basement, caused by flooding, which Union denied citing a policy exclusion for loss caused by water damage, including floods and surface water. Union argued the damage resulted from flood and surface water, while the plaintiffs contended the water was not surface water and accused Union of shifting its defense inappropriately under the "mend the hold" doctrine. The trial court granted summary judgment in favor of Union, rejecting plaintiffs' arguments, including their claim under the "mend the hold" doctrine, which prevents a party from changing the reason for breach of contract once litigation has begun. The trial court's decision, based on the pleaded water damage exclusion and expert testimonies on the cause of the damage, highlighted the non-applicability of the "mend the hold" doctrine since Union did not change the foundational basis of its defense but rather specified a type of water contributing to the damage. This case underscores the necessity of a clear understanding of insurance policy exclusions and the challenges in litigating insurance claims involving nuanced definitions of excluded perils.
URL: https://api.case.law/v1/cases/171775/

Instructions: Using only the provided search results that are relevant, and starting with the most relevant, write a detailed comparative analysis for a new query. If there are no relevant cases say so, and use one example from the search results to illustrate the uniqueness of the new query. ALWAYS cite search results using [[number](URL)] notation after the reference.

New Query:
How have courts interpreted the phrase 'sudden and accidental' in the context of water damage claims, and what are the key legal precedents and judicial decisions that have influenced this interpretation?

Analysis:

In [19]:
report = get_final_answer(
    formatted_input=formatted_input,
    model_name='gpt-4-turbo',
    )
Markdown(report.research_report)

The phrase 'sudden and accidental' has been a central issue in several insurance policy disputes concerning environmental and other types of damage. Various cases provide insights into how courts interpret this phrase, particularly in the context of water damage claims. 

In the first case, the interpretation of 'sudden and accidental' is disputed in the context of environmental contamination, where plaintiffs argue that under Illinois law, the term should mean 'unexpected and unintended,' contrasting with 'abrupt' under Missouri law [[1](https://api.case.law/v1/cases/5454708/)]. This ongoing debate over the jurisdictional application and definitions underlines the significant difference in interpretations that can affect the outcome of litigation involving insurance coverage for environmental damages.

Another relevant case involves a dispute over whether a sewage system explosion, falling under the exception for explosions despite prior exclusions for water damage, should be considered 'sudden and accidental' [[2](https://api.case.law/v1/cases/12526556/)]. This case illustrates the complexity of separating cause and effect in insurance disputes and the importance of clearly understanding policy terms in determining coverage.

In the third relevant case, the reliability of expert testimony was critical in deciding whether chemical leaks were 'sudden and accidental.' The court's decision to disregard the expert's testimony due to its speculative nature and lack of reliable methodologies reflects the scrutiny legal arguments undergo in such disputes [[3](https://api.case.law/v1/cases/12521187/)].

These cases collectively emphasize the varied judicial interpretations of 'sudden and accidental,' which often hinge on specific facts and expert testimonies, as well as the nuanced understanding of policy language. They reveal that 'sudden and accidental' often involves a detailed analysis of the circumstances surrounding the incident and the policy wording that can shift the balance in judicial decisions.

In [20]:
report.citations

['1', '2', '3']

In [16]:
from src.agent.tools.utils import convert_timestamp_to_datetime
from src.utils.citations import create_citation_lookup_table, get_top_citations, get_citation_context

In [17]:
prepped_context.head(1)

,context,index,id,citation,name,name_abbreviation,decision_date,court_id,court_name,court_slug,judges,attorneys,citations,url,head,body,name_contains_lm,body_contains_lm,year,context_citation,context_tokens,state,search_type,score,summary_analysis,relevance_score
0,Id. at 4. According to Plaintiff's insurance p...,2126,12526556,91 L. Ed. 2d 265,"Audrey SANKO, Plaintiff, v. ALLSTATE INSURANCE...",Sanko v. Allstate Ins. Co.,2018-05-18,9039,United States District Court for the Middle Di...,md-pa,[],"['Tammy Lee Clause, Attorney Tammy Lee Clause,...","[{'type': 'official', 'cite': '323 F. Supp. 3d...",https://api.case.law/v1/cases/12526556/,"Audrey SANKO, Plaintiff,\r\nv.\r\nALLSTATE INS...","Robert D. Mariani, United States District Judg...",False,True,2018,91 L. Ed. 2d 265,1442,Wyoming,vector,0.976563,The excerpt discusses a dispute over an insura...,9


In [18]:
test_df = prepped_context[['id', 'name', 'url', 'body', 'state', 'decision_date', 'summary_analysis']]

In [19]:
citation_table = create_citation_lookup_table(test_df, text_column='body', id_column='id')

In [20]:
citation_table['citation'].nunique()

252

In [21]:
top_citations = get_top_citations(df=test_df, text_column='body', top_n=5)
Markdown(top_citations.to_markdown(index=False))

| Citation        |   Count | link                                 |
|:----------------|--------:|:-------------------------------------|
| 154 Ill. 2d 90  |       6 | https://cite.case.law/ill-2d/154/90  |
| 197 Ill. 2d 278 |       5 | https://cite.case.law/ill-2d/197/278 |
| 144 Ill. 2d 64  |       5 | https://cite.case.law/ill-2d/144/64  |
| 607 N.E.2d 1204 |       3 | https://cite.case.law/ne2d/607/1204  |
| 757 N.E.2d 481  |       3 | https://cite.case.law/ne2d/757/481   |

In [22]:
top_cites = top_citations['Citation'].unique().tolist()


In [43]:
from src.utils.gen_utils import count_tokens

def create_citation_context(
    df: pd.DataFrame,
    citation: str,
    text_column: str,
    date_column: str,
    words_before: int = 200,
    words_after: int = 200,
    context_token_limit: int = 3000,
) -> str:
    """Creates a citation context string from a DataFrame based on the provided columns.

    Args:
        df (pd.DataFrame): The DataFrame containing the citation data.
        citation (str): The target citation string.
        text_column (str): The name of the column containing the text data.
        name_column (str): The name of the column containing the case names.
        date_column (str): The name of the column containing the decision dates.
        words_before (int, optional): The number of words to include before the citation. Defaults to 200.
        words_after (int, optional): The number of words to include after the citation. Defaults to 200.
        context_token_limit (int, optional): The maximum number of tokens allowed in the context. Defaults to 3000.

    Returns:
        str: The citation context string created from the DataFrame.
    """
    citation_context = ""
    total_tokens = 0

    for _, row in df.iterrows():
        text = row[text_column]
        date = row[date_column].strftime("%Y-%m-%d")

        cite_context = get_citation_context(
            text=text,
            citation=citation,
            words_after=words_after,
            words_before=words_before,
        )

        if cite_context != "":
            context_entry = f"Citation: **{citation}**\n\n"
            context_entry += f"Context Date: {date}\n\n"
            context_entry += f"Context: {cite_context}\n\n"
            context_entry += "-" * 100 + "\n\n"

            entry_tokens = count_tokens(context_entry)
            if total_tokens + entry_tokens > context_token_limit:
                break

            citation_context += context_entry
            total_tokens += entry_tokens

    return citation_context

In [26]:
Markdown(cite_context_list[0])

Citation: **154 Ill. 2d 90**

Case Name: EMERSON ELECTRIC COMPANY et al., Plaintiffs-Appellants, v. AETNA CASUALTY AND SURETY COMPANY et al., Defendants-Appellees, decided on 2004-08-30

Context: Summary judgment is appropriate where “the pleadings, depositions, and admissions on file, together with the affidavits, if any, show that there is no genuine issue as to any material fact and that the moving party is entitled to a judgment as a matter of law.” 735 ILCS 5/2—1005(c) (West 2002). Orders granting summary judgment are reviewed de novo. Rumford v. Countrywide Funding Corp., 287 Ill. App. 3d 330, 334, 678 N.E.2d 369, 372 (1997). On appeal from such an order, the reviewing court “must independently examine the evidence presented in support of and in opposition to a motion for summary judgment.” Groce v. South Chicago Community Hospital, 282 Ill. App. 3d 1004, 1006, 669 N.E.2d 596, 598 (1996). Where a reasonable person could draw divergent inferences from undisputed facts, summary judgment is improper. Outboard Marine Corp. v. Liberty Mutual Insurance Co., 154 Ill. 2d 90, 102, 607 N.E.2d 1204, 1209 (1992). If, from a review of the pleadings and evidentiary material before the circuit court, this court determines that a material issue of fact exists or that the summary judgment was based upon an erroneous interpretation of the law, a reversal is warranted. Metropolitan Life Insurance Co. v. American National Bank & Trust Co., 288 Ill. App. 3d 760, 765, 682 N.E.2d 72, 75 (1997). Construction of an insurance policy is a question of law and, therefore, may appropriately be disposed of through summary judgment. See American Standard Insurance Co. v. Allstate Insurance Co., 210 Ill. App. 3d 443, 446, 569 N.E.2d 162, 165 (1991). In construing the language of the policy, the court’s primary task is “to ascertain and give effect to the intent of the parties to the contract.” Traveler’s Insurance Co. v. Eljer Manufacturing, Inc., 197 Ill. 2d 278, 292, 757 N.E.2d 481, 491 (2001). In order to ascertain the meaning of the policy’s language and the parties’ intent, the court must construe the policy as a whole and take into account the type of insurance purchased, the nature of the risks involved, and the overall purpose of the contract.

----------------------------------------------------------------------------------------------------

Citation: **154 Ill. 2d 90**

Case Name: STONERIDGE DEVELOPMENT COMPANY, INC., et al., Plaintiffs and Counterdefendants-Appellees, v. ESSEX INSURANCE COMPANY, Defendant and Counterplaintiff and Counterdefendant-Appellant (Residential Warranty Corporation et al., Defendants and Counterdefendants and Counterplaintiffs-Appellees), decided on 2008-05-06

Context: Accordingly, we conclude that we have jurisdiction over this appeal. B. Estoppel Turning to the merits, Essex first argues that the trial court erred by granting appellees summary judgment based on estoppel. Summary judgment is appropriate only where the pleadings, affidavits, depositions, admissions, and exhibits on file, when viewed in the light most favorable to the nonmoving party, show that there is no genuine issue of material fact and that the moving party is entitled to judgment as a matter of law. Zekman v. Direct American Marketers, Inc., 182 Ill. 2d 359, 374 (1998). We review de novo a grant of summary judgment. Rich v. Principal Life Insurance Co., 226 Ill. 2d 359, 370 (2007). Also, the construction of an insurance policy is a question of law, to which de novo review applies. Rich, 226 Ill. 2d at 371. We begin by discussing the relevant case law on an insurer’s duty to defend. An insurer’s duty to defend its insured is much broader than its duty to indemnify the insured. Outboard Marine Corp. v. Liberty Mutual Insurance Co., 154 Ill. 2d 90, 125 (1992). A duty to indemnify arises only if the insured has a judgment against it on any underlying claim and the insured’s activity and the resulting loss or damage actually come within the policy’s coverage. Outboard Marine Corp., 154 Ill. 2d at 127-28. In contrast, the precursor duty to defend arises if, liberally construing in the insured’s favor the allegations in the underlying complaint against the insured, there are factual allegations that even potentially fall within the coverage. General Agents Insurance Co. of America, Inc. v. Midwest Sporting Goods Co., 215 Ill. 2d 146, 154-55 (2005). Moreover, if the underlying complaint against the insured contains several theories of recovery and only one of the theories is potentially covered, the insurer must still defend the insured. Midwest Sporting Goods Co., 215 Ill. 2d at 155. In this manner, the insurer may become obligated to defend against causes of action and theories of recovery that the policy does not actually cover. Illinois Masonic Medical Center v. Turegum Insurance Co., 168 Ill. App. 3d 158, 162 (1988).

----------------------------------------------------------------------------------------------------

Citation: **154 Ill. 2d 90**

Case Name: MARY NOREN et al., Plaintiffs-Appellees, v. METROPOLITAN PROPERTY AND CASUALTY INSURANCE COMPANY, Defendant-Appellant, decided on 2006-12-06

Context: Only then can a proper determination be made as to whether the damage was covered under the policy. Because the court erred in striking defendant’s jury demand, we remand this cause to the circuit court for a new trial. In light of this determination, we need only address one of defendant’s remaining contentions on appeal, that the court erred in denying its motion for summary judgment. Summary judgment is proper if the pleadings, depositions, affidavits, admissions, and other matters on file demonstrate that there is no genuine issue of material fact and that the movant is entitled to judgment as a matter of law. Smith v. Armor Plus Co., 248 Ill. App. 3d 831, 839 (1993). The court should construe the evidence strictly against the movant and liberally in favor of the opponent. Richter v. Burton Investment Properties, Inc., 240 Ill. App. 3d 998, 1001 (1993). If reasonable persons could draw different inferences from undisputed facts, an issue of fact exists. Armor, 248 Ill. App. 3d at 839. Appellate review of an order granting summary judgment is de novo. Outboard Marine Corp. v. Liberty Mutual Insurance Co., 154 Ill. 2d 90, 102 (1992). Here, as stated above, there were numerous questions of fact the circuit court was required to make because there was conflicting evidence as to the cause of the water damage to plaintiffs’ property. Defendant argues in its brief that because plaintiffs presented no evidence regarding the cause of their property damage, there were no issues of material fact. However, we point out that in defendant’s brief, in support of its argument that the court erred in striking its jury demand, defendant states that there were factual issues as to how the property damage occurred. In fact, defendant argued in its reply brief that there were a “litany of factual issues.” We are unable to reconcile defendant’s inconsistent positions. It is illogical for defendant to argue on the one hand that there were no factual issues for summary judgment purposes and on the other hand to argue that there were many factual issues for jury demand purposes. Defendant cannot have it both ways. We find that the circuit court properly denied defendant’s motion for summary judgment because a question of fact existed as to how plaintiffs’ property was damaged.

----------------------------------------------------------------------------------------------------



In [55]:
import instructor
import openai
from tenacity import Retrying, stop_after_attempt, wait_fixed

from src.search.models import CitationInformation

def analyze_citation(
    citation: str, excerpts: str
) -> CitationInformation:
    client = instructor.patch(openai.OpenAI())
    return client.chat.completions.create(
        model="gpt-4-turbo-preview",
        seed=42,
        response_model=CitationInformation,
        max_retries=Retrying(
            stop=stop_after_attempt(5),
            wait=wait_fixed(1),
        ),
        messages=[
            {
                "role": "system",
                "content": "Your role is to analyze the given Context, which is previously extracted information related to a legal case law citation, and distill down the main takeaways using the correct format.",
            },
            {
                "role": "user",
                "content": f"Your task focuses solely on citation: **{citation}**",
            },
            {   "role": "user", "content": f"Here is the Context: {excerpts}"},
        ],
    )

In [56]:
def get_top_citations_with_context(df: pd.DataFrame, text_column: str, date_column: str, top_n: int = 5):
    df.sort_values(by=date_column, inplace=True)
    top_citations = get_top_citations(df=df, text_column=text_column, top_n=top_n)
    top_cites = top_citations['Citation'].unique().tolist()

    cite_context_list = []

    for cite in top_cites:
        res = create_citation_context(
            df=df,
            citation=cite,
            text_column=text_column,
            date_column=date_column,
            context_token_limit=3000,
        )
        cite_context_list.append(res)
        
    return top_cites, cite_context_list

In [53]:
top_citations, top_citations_context = get_top_citations_with_context(test_df, 'body', 'decision_date')

In [57]:
cite_summary = analyze_citation(
    citation=top_citations[0],
    excerpts=top_citations_context[0]
)

In [61]:
print(cite_summary)

Citation: 154 Ill. 2d 90
Summary:
The citation '154 Ill. 2d 90' primarily relates to principles around summary judgment and
construction of insurance policies in the context of legal disputes. It emphasizes the de novo
standard of review for summary judgment and insurance policy interpretation, highlighting the
court's role in ascertaining the intent of the parties involved through policy documents. The case
underscores that summary judgment is suitable only when there are no genuine issues of material fact
and the moving party is entitled to judgment as a matter of law. It also delves into the insurer's
duty to defend, which is based on whether the allegations in the underlying complaint potentially
fall within the policy's coverage, suggesting that judicial interpretation of policy language must
consider the whole policy in the context of the contract's overall purpose.
Questions:
- What is the standard of review for summary judgment decisions in insurance policy disputes?
- How shou

In [54]:
Markdown(top_citations_context[0])

Citation: **154 Ill. 2d 90**

Context Date: 2003-02-11

Context: In response, on October 11, 2001, the plaintiff filed a motion to stay summary judgment proceedings and to compel discovery. On October 24, 2001, the circuit court entered an order continuing generally the plaintiffs motion to stay and compel discovery. On November 26, 2001, the plaintiff filed a response to the defendant’s motion for partial summary judgment and a cross-motion for partial summary judgment. On February 1, 2002, the circuit court issued a memorandum and order granting the defendant’s motion for partial summary judgment as to count I of the complaint. On February 7, 2002, the defendant filed a motion for partial summary judgment as to count II of the complaint. The motion was granted by the circuit court on February 19, 2002. The plaintiff filed a timely notice of appeal. ANALYSIS I. Standard of Review The court reviews the granting of motions for summary judgment de novo. Outboard Marine Corp. v. Liberty Mutual Insurance Co., 154 Ill. 2d 90, 102, 607 N.E.2d 1204, 1209 (1992). II. Policy Construction The construction of an insurance policy and a determination of the rights and obligations thereunder are questions of law for the court that are appropriate subjects for disposition by way of summary judgment. Crum & Forster Managers Corp. v. Resolution Trust Corp., 156 Ill. 2d 384, 391, 620 N.E.2d 1073, 1077 (1993). In construing an insurance policy, the primary function of the court is to ascertain and enforce the intention of the parties as expressed in the agreement. Crum & Forster Managers Corp., 156 Ill. 2d at 391, 620 N.E.2d at 1078. To ascertain the intent of the parties and the meaning of the words used in the insurance policy, the court must construe the policy as a whole, taking into account the type of insurance for which the parties have contracted, the risks undertaken and purchased, the subject matter that is insured and the purposes of the entire contract. Crum & Forster Managers Corp., 156 Ill. 2d at 391, 620 N.E.2d at 1078. If the words of the policy are plain and unambiguous, the court will afford them their plain, ordinary meaning and will apply them as written.

----------------------------------------------------------------------------------------------------

Citation: **154 Ill. 2d 90**

Context Date: 2004-08-30

Context: Summary judgment is appropriate where “the pleadings, depositions, and admissions on file, together with the affidavits, if any, show that there is no genuine issue as to any material fact and that the moving party is entitled to a judgment as a matter of law.” 735 ILCS 5/2—1005(c) (West 2002). Orders granting summary judgment are reviewed de novo. Rumford v. Countrywide Funding Corp., 287 Ill. App. 3d 330, 334, 678 N.E.2d 369, 372 (1997). On appeal from such an order, the reviewing court “must independently examine the evidence presented in support of and in opposition to a motion for summary judgment.” Groce v. South Chicago Community Hospital, 282 Ill. App. 3d 1004, 1006, 669 N.E.2d 596, 598 (1996). Where a reasonable person could draw divergent inferences from undisputed facts, summary judgment is improper. Outboard Marine Corp. v. Liberty Mutual Insurance Co., 154 Ill. 2d 90, 102, 607 N.E.2d 1204, 1209 (1992). If, from a review of the pleadings and evidentiary material before the circuit court, this court determines that a material issue of fact exists or that the summary judgment was based upon an erroneous interpretation of the law, a reversal is warranted. Metropolitan Life Insurance Co. v. American National Bank & Trust Co., 288 Ill. App. 3d 760, 765, 682 N.E.2d 72, 75 (1997). Construction of an insurance policy is a question of law and, therefore, may appropriately be disposed of through summary judgment. See American Standard Insurance Co. v. Allstate Insurance Co., 210 Ill. App. 3d 443, 446, 569 N.E.2d 162, 165 (1991). In construing the language of the policy, the court’s primary task is “to ascertain and give effect to the intent of the parties to the contract.” Traveler’s Insurance Co. v. Eljer Manufacturing, Inc., 197 Ill. 2d 278, 292, 757 N.E.2d 481, 491 (2001). In order to ascertain the meaning of the policy’s language and the parties’ intent, the court must construe the policy as a whole and take into account the type of insurance purchased, the nature of the risks involved, and the overall purpose of the contract.

----------------------------------------------------------------------------------------------------

Citation: **154 Ill. 2d 90**

Context Date: 2006-12-06

Context: Only then can a proper determination be made as to whether the damage was covered under the policy. Because the court erred in striking defendant’s jury demand, we remand this cause to the circuit court for a new trial. In light of this determination, we need only address one of defendant’s remaining contentions on appeal, that the court erred in denying its motion for summary judgment. Summary judgment is proper if the pleadings, depositions, affidavits, admissions, and other matters on file demonstrate that there is no genuine issue of material fact and that the movant is entitled to judgment as a matter of law. Smith v. Armor Plus Co., 248 Ill. App. 3d 831, 839 (1993). The court should construe the evidence strictly against the movant and liberally in favor of the opponent. Richter v. Burton Investment Properties, Inc., 240 Ill. App. 3d 998, 1001 (1993). If reasonable persons could draw different inferences from undisputed facts, an issue of fact exists. Armor, 248 Ill. App. 3d at 839. Appellate review of an order granting summary judgment is de novo. Outboard Marine Corp. v. Liberty Mutual Insurance Co., 154 Ill. 2d 90, 102 (1992). Here, as stated above, there were numerous questions of fact the circuit court was required to make because there was conflicting evidence as to the cause of the water damage to plaintiffs’ property. Defendant argues in its brief that because plaintiffs presented no evidence regarding the cause of their property damage, there were no issues of material fact. However, we point out that in defendant’s brief, in support of its argument that the court erred in striking its jury demand, defendant states that there were factual issues as to how the property damage occurred. In fact, defendant argued in its reply brief that there were a “litany of factual issues.” We are unable to reconcile defendant’s inconsistent positions. It is illogical for defendant to argue on the one hand that there were no factual issues for summary judgment purposes and on the other hand to argue that there were many factual issues for jury demand purposes. Defendant cannot have it both ways. We find that the circuit court properly denied defendant’s motion for summary judgment because a question of fact existed as to how plaintiffs’ property was damaged.

----------------------------------------------------------------------------------------------------

Citation: **154 Ill. 2d 90**

Context Date: 2007-03-19

Context: This appeal followed. II. ANALYSIS Carr argues the third amended complaint in the underlying case (case No. 04—L—8) alleges an “occurrence” covered by his CGL policy through Country Mutual. A. Standard of Review The standard of review for reviewing a trial court’s decision to grant a motion for judgment on the pleadings is de novo. Board of Trustees of the University of Illinois v. City of Chicago, 317 Ill. App. 3d 569, 571, 740 N.E.2d 515, 517 (2000). The same is true for reviewing a trial court’s interpretation of an insurance policy. Travelers Insurance Co. v. Eljer Manufacturing, Inc., 197 Ill. 2d 278, 292, 757 N.E.2d 481, 491 (2001). In this appeal, we must determine whether the trial court erred in finding the complaint in the underlying case did not allege an “occurrence.” B. Insurance Company’s Duty To Defend The underlying issues in case No. 04—L—15 from which this appeal arises are whether Country Mutual has a duty to defend and possibly indemnify Carr in case No. 04—L—8. An insurance company’s duty to defend its policyholder is much broader than its duty to indemnify that same policyholder. Outboard Marine Corp. v. Liberty Mutual Insurance Co., 154 Ill. 2d 90, 125, 607 N.E.2d 1204, 1220 (1992). To determine whether an insurance company has a duty to defend, the court must compare the allegations in the underlying complaint to the language of the insurance policy at issue. Outboard Marine Corp., 154 Ill. 2d at 125, 607 N.E.2d at 1220. In so doing, the court must give the allegations in the complaint a liberal construction in favor of the insured. Outboard Marine Corp., 154 Ill. 2d at 125, 607 N.E.2d at 1220. Further, the court must liberally construe the provisions of the insurance policy in favor of coverage. State Security Insurance Co. v. Burgos, 145 Ill. 2d 423, 438, 583 N.E.2d 547, 554 (1991). After giving both the allegations in the complaint and the provisions of the policy a liberal construction in favor of the insured having coverage, the court must determine if the “allegations fall within, or potentially within, the policy’s coverage.” (Emphasis in original.) Outboard Marine Corp., 154 Ill. 2d at 125, 607 N.E.2d at 1220. If the court finds this to be so, “the insurer has a duty to defend the insured against the underlying complaint.” Outboard Marine Corp., 154 Ill. 2d at 125, 607 N.E.2d at 1220.

----------------------------------------------------------------------------------------------------

Citation: **154 Ill. 2d 90**

Context Date: 2008-05-06

Context: Accordingly, we conclude that we have jurisdiction over this appeal. B. Estoppel Turning to the merits, Essex first argues that the trial court erred by granting appellees summary judgment based on estoppel. Summary judgment is appropriate only where the pleadings, affidavits, depositions, admissions, and exhibits on file, when viewed in the light most favorable to the nonmoving party, show that there is no genuine issue of material fact and that the moving party is entitled to judgment as a matter of law. Zekman v. Direct American Marketers, Inc., 182 Ill. 2d 359, 374 (1998). We review de novo a grant of summary judgment. Rich v. Principal Life Insurance Co., 226 Ill. 2d 359, 370 (2007). Also, the construction of an insurance policy is a question of law, to which de novo review applies. Rich, 226 Ill. 2d at 371. We begin by discussing the relevant case law on an insurer’s duty to defend. An insurer’s duty to defend its insured is much broader than its duty to indemnify the insured. Outboard Marine Corp. v. Liberty Mutual Insurance Co., 154 Ill. 2d 90, 125 (1992). A duty to indemnify arises only if the insured has a judgment against it on any underlying claim and the insured’s activity and the resulting loss or damage actually come within the policy’s coverage. Outboard Marine Corp., 154 Ill. 2d at 127-28. In contrast, the precursor duty to defend arises if, liberally construing in the insured’s favor the allegations in the underlying complaint against the insured, there are factual allegations that even potentially fall within the coverage. General Agents Insurance Co. of America, Inc. v. Midwest Sporting Goods Co., 215 Ill. 2d 146, 154-55 (2005). Moreover, if the underlying complaint against the insured contains several theories of recovery and only one of the theories is potentially covered, the insurer must still defend the insured. Midwest Sporting Goods Co., 215 Ill. 2d at 155. In this manner, the insurer may become obligated to defend against causes of action and theories of recovery that the policy does not actually cover. Illinois Masonic Medical Center v. Turegum Insurance Co., 168 Ill. App. 3d 158, 162 (1988).

----------------------------------------------------------------------------------------------------



In [6]:
import os
from dotenv import load_dotenv
import serpapi
load_dotenv()

client = serpapi.Client(api_key=os.getenv("SERPER_API_KEY"))

params = {
  "engine": "home_depot",
  "q": "1/2 inch drywall",
}

results = client.search(params)

In [9]:
results

{
    "search_metadata": {
        "id": "66371a0bb7b1ccd7df1ef3b9",
        "status": "Success",
        "json_endpoint": "https://serpapi.com/searches/412ba76036ab84ea/66371a0bb7b1ccd7df1ef3b9.json",
        "created_at": "2024-05-05 05:32:59 UTC",
        "processed_at": "2024-05-05 05:32:59 UTC",
        "home_depot_url": "https://www.homedepot.com/b/N-5yc1v/Ntt-1%2F2+inch+drywall?Nao=0",
        "raw_html_file": "https://serpapi.com/searches/412ba76036ab84ea/66371a0bb7b1ccd7df1ef3b9.html",
        "prettify_html_file": "https://serpapi.com/searches/412ba76036ab84ea/66371a0bb7b1ccd7df1ef3b9.prettify",
        "total_time_taken": 2.5
    },
    "search_parameters": {
        "q": "1/2 inch drywall",
        "nao": "0",
        "ps": 24,
        "delivery_zip": "04401",
        "store_id": "2414",
        "country": "us",
        "engine": "home_depot"
    },
    "search_information": {
        "results_state": "Results for exact spelling",
        "total_results": 91,
        "store